<a href="https://colab.research.google.com/github/Knightler/deep-learning/blob/main/PYTORCH_Paper_Replicating_Project02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 08. PyTorch Paper Replicating

The goal of machine learning research paper replicating is to turn a ML research paper into usable code.

In this notebook we're going to be replicating the Vision Transformer (ViT) architecture/paper with PyTorch: https://arxiv.org/abs/2010.11929

In [ ]:
## 0. Get setup

Let's import code we've previously written